## Рекомендательные системы: товары, которые больше понравятся

Делаем рекомендации категорий товаров в интернет-магазине.

Для каждого пользователя известно, когда и по товару какой категории он кликал.


#### Метрика качества - mean average precision@k. 
Посмотрим детальнее на метрику: 
- precision@k - доля купленного из рекомендованного

- average precision@k - внутри одного пользователя (одной рекомендации) усредняем precision@k по позициям k (так решаем проблему предыдущей метрики - она не учитывала порядок элементов в "топе")

- mean average precision@k - усредняем по пользователям


In [ ]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.

    This function computes the average prescision at k between two lists of
    items.

    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The average precision at k over the input lists

    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.

    This function computes the mean average prescision at k between two lists
    of lists of items.

    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements

    Returns
    -------
    score : double
            The mean average precision at k over the input lists

    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

**Задача:** хотим предсказывать, какая категория товара пользователю будет интересна на оснве информации о кликах. 
Т.е. построить модель, предсказываюшую вероятность клика.

**Общая схема построение модели:**

- обучение модели: объектом выборки будет являться пара (user_id, category_id). Таргет - "клик"/"не клик", т.е. бинарная классификация. Будем обучать бустинг на logloss, чтобы получить вероятность, по которой в дальнейшем ранжировать.

- здесь надо подумать о том, что мы будем считать "не кликом" (негативным классом), ведь у нас етсь только информация о кликах. Будем выбирать рандомно **несколько** негативных примеров из всех оставшихся категорий, по которые пользователь не кликнул). (**несколько**, а не все возможные, потому что в противном случае получится сильный дисбаланс классов)

- предсказание: для каждого пользователя из тестовой выборки и каждой возможной категории генерим признаки.

- принимаем решение согласно модели: выберем топ 5 категорий для каждого пользователя по полученной вероятности.


In [ ]:
import numpy as np
import pandas as pd
from scipy import sparse
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

import random

In [ ]:
clicks = pd.read_csv('clicks.csv', sep=',')


**1.** Разбейте данные на три части: 
- все до предпоследней недели (по этой неделе будем считать фичи о пользователе, о товаре, взаимные фичи)
- предпоследняя неделя (по этой неделе сгенерируем обучающую выборку) 
- последняя неделя (тест, на котором смотрим качество -- пользователи, для которых хотим предсказать. Замечание: разумеется, здесь есть некоторая "утечка": на практике возможно предсказывать для тех пользователей, которые 1) пришли до момента предсказания 2) обычно мы не знаем, кто из них появится в нашей системе на интересующей неделе). В этой задаче мы не будем обращать на это внимание.

In [ ]:
pred_last_week_start = '2016-09-12' 
last_week_start = '2016-09-19' 

In [ ]:
stat_clicks = #
train_clicks = #
test_clicks = #

In [ ]:
clicks.head()

**2.** По stat_clicks хотим посчитать статистики для фичей о пользователях и товарах.

- создайте в датафрейме столбец 'list', в котором будет лежать лист из "троек" (user_id, category_id, day)
- составим обучающую выборку (сами объекты, пока что без признакового описания): для каждого пользователя возьмите категории, которые входили в его топ-5 кликнутых (это будет класс 1), и еще 5 случайных категорий (класс 0).

Чем плох следующий вариант: для всех категорий, которые не просмотрел пользователь, поставить класс 0?

In [ ]:
stat_clicks['list'] = #

In [ ]:
stat_clicks.loc[1].list

In [ ]:
categories = np.unique(clicks.category_id.values)

# функция возврашает лейблы 1/0 в y (здесь же сэмплируем негативные примеры)
# айди категории, айди пользователя в X 

def get_sample(train_clicks_list):
    sample, target = [], []

    return target, sample
    
train_clicks_list = train_clicks.groupby('user_id')['category_id'].apply(list)   
targets, objects = get_sample(train_clicks_list)

**3.**
Напишите код, который считает вспомогательные статистики по пользователям и категориям.


Признаки по категории:
- сколько всего кликов было по этой категории
- сколько уникальных пользователей кликало по этой категории
- сколько категорий встречалось с конкретной категорией (внутри одного пользователя) = топ 5 категорий, которые встречались вместе с конкретной внутри одного пользователя

По пользователю:
- для каждого пользователя список всех категорий, по которым он кликал (затем каждую из этих категорий будем кодировать числом - суммарным числом кликов по ней )
- посчитаем топ 5 самых популярных категорий (по всей выборке), затем посмотрим, кликал ли пользователь по этим 5 популярным (это будет 5 фичей для каждого пользователя = (просматривал ли категории из топ 5 когда-либо))

In [ ]:
from collections import defaultdict, Counter

In [ ]:
# по пользователю
categories_on_users = defaultdict(list)

# YOUR CODE IS HERE

# 5 самых поулярных категорий
popular_cats = #

# по категориям
# сколько всего кликов было по этой категории
clicks_on_category = Counter()
# сколько уникальных пользователей кликало по этой категории
users_on_category = defaultdict(set)
# сколько категорий встречалось с конкретной категорией (внутри одного пользователя)
category_with_category = defaultdict(list) 

# YOUR CODE IS HERE

**4.** Для каждого объекта обучающей выборке (objects) создайте следующие признаки:
- сколько было кликов по этой категории
- сколько было различных пользователей, кликнувших по этой категори
- топ 5 которые встречались с данной категорией category_with_category_cnt: их кодируем встречаемостью всех кликов
- топ 5 самых крутых категорий по пользователю, закодированных числом кликов по ней со всей выборки
- 5 OHE-фичей (кликал ли пользователь 5 популярным)

Обучите xgboost.

In [ ]:
X_train_feats = []

for cat_id, user_id in objects:
    
    features_train  = []
    # фичи по категории
    # YOUR CODE IS HERE
    features_train.append()
    # фичи по пользователям
    # YOUR CODE IS HERE
    X_train_feats.append(features_train)
    
X_train_feats = np.array(X_train_feats)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier().fit(X_train_feats, np.array(targets))

**5.** Теперь сделаем предсказание для каждого user_id  и каждой категории.
- создайте список всех категорий, для которой мы хотим предсказывать (внутри каждого пользователя)
- напишите функцию, которая по user_id будет генерировать признаковое описание для каждой возможной категории
- напишите функцию,которая по user_id будет генерировать предсказание

In [ ]:
test_clicks_list = test_clicks.groupby('user_id')['category_id'].apply(list)

In [ ]:
# все категории, для которой мы хотим предсказывать (внутри каждого пользователя)
categories_to_pred = 

In [ ]:
def get_features(user_id, cat_id):

    features_train  = []
    # YOUR CODE IS HERE
    
    return features_train

In [ ]:
def get_prediction(user_id):
    return 

import heapq
# для оптимизации использовать heapq
# heapq.nlargest(5, zp(categories_to_pred, pred), key=1)

In [ ]:
get_prediction(100)

**6.** Смотрим качество:

In [ ]:
m = []
for user_id, arr in zip(test_clicks_list.index, test_clicks_list)[1800:2000]:
    actual = test_clicks_list.loc[user_id]
    predicted = get_prediction(user_id)
    m.append(apk(actual, predicted, 5))

In [ ]:
np.mean(m)

Какие фичи еще можно исопльзовать?

- временные: день недели играет большую роль (в выходные и праздничные дни пользователи ведут себя не так, как в будние)
- признаки по пользователю и категории с учетом времени (просматривал ли пользователь эту категорию неделю назад, 2 недели назад и т.д.)